# Tareas
1) Desde un notebook, script para llamar a ecobici y guardar alguna info SIMPLE (ej: cuántos slots ocupados) en csv.
2) Agregar el secret para usar en github actions
3) Configurar la Github Action para que haga un commit (en una branch en particular? O en main?)
4) Luego de probar que funciona, agregar el cron a la github action
5) Mergear a main el nuevo código

In [1]:
import pandas as pd

from internal.platform.environment.environment import set_environment
from internal.gateway.transportBA import transportBA
from internal.repository.transportBA.stations_repository import StationsRepository

In [2]:
set_environment()

In [3]:
def preprocessStations(stations_df):
    stations_df = stations_df[["station_id", "groups"]]
    return stations_df.explode("groups")

def preprocessStationsStatus(stations_status_df):
    stations_status_df["status_last_reported_utc"] = pd.to_datetime(stations_status_df["last_reported"], unit="s", utc=True)
    return station_status_df[
        ["station_id",
         "num_bikes_available", 
         "num_bikes_disabled", 
         "num_docks_available", 
         "num_docks_disabled",
         "status_last_reported_utc"
         ]
    ]

In [4]:
transportBAClient = transportBA.Client()
stations_df = transportBAClient.GetStations()
station_status_df = transportBAClient.GetStationStatus()
stationsDao = StationsRepository()

stations_df = preprocessStations(stations_df)
station_status_df = preprocessStationsStatus(station_status_df)
join = pd.merge(stations_df, station_status_df, on="station_id")

stationsDao.AppendWithJobRunAt(join)

HTTPError: 401 Client Error: Unauthorized for url: https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation